In [ ]:
import os
import sys
import re

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

import init_titration
from titration import util as utl
from mfethuls import parse as pa     # Import your own parser if mfethuls not used

### Generate titrations from the config file

In [ ]:
# Choose one of the two exp_name list to plot data. Comment out the one that is not of interest.

# exp_names = ['LUB005', 'LUB021', 'LUB038', 'LUB192'] # Zn(EH-Mebip)2 decomplex with different ACIDS + TEAC
exp_names = ['LUB045', 'LUB005', 'LUB157', 'LUB067', 'LUB055'] # Different METALS of MEBIP complex - decomplex with HCL

which_data = 'metals' if 'LUB157' in exp_names else None # Allows for change in configuration for graphing

In [ ]:
init_titration.init_titrations(exp_names)

In [ ]:
dict_titrants_exps = init_titration.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(init_titration.dict_titrations_exps)
dict_titrants_exps

In [ ]:
# Show the dataframe housing all
for k, v in dict_titrations_exps.items():
    print(f'\n\n{k}:\n')
    for t in v[0]:
        print(t.df_params)

### Import data associated with experiment

In [ ]:
exps_reps = utl.get_exp_reps(exp_names)
data = pa.get_data(pa.path_constructor('uv', *exps_reps), 'uv')
data

### Distinguish from different titrations and merge titration params with data imported

In [ ]:
new_data = utl.merge_data_titration_params(data, dict_titrations_exps)
new_data

In [ ]:
new_data.titrant.unique()

### Generate peak tracking data at specified wavelengths

In [ ]:
# wls=335 nm for Zn(EH-Mebip)2 [MLCT]
# wl_0=313 for EH-Mebip [pi-pi*]
# wls=[335, 350, 345, 345, 350, 350, 350] if metals chosen

wls = [335, 350, 345] if which_data else [335]
dict_df = utl.tracking_df(new_data, wls, exps_reps, wl_0=313, complex_tit=True)
dict_df

In [ ]:
# Combine tracking data with the original dataframe of experiment
df = utl.combine_track_data(new_data, dict_df)
df

In [ ]:
# For different metals only keep relevant MLCT peak tracks
if which_data:
    
    # Name and associated mlct band wl for experiment
    labels_MLCT = dict(zip(['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157'], 
                           [335, 350, 345, 345, 350])) # All Mebip complexes MLCT with different metals
    
    df = utl.remove_unrelated_wl_track(df, labels_MLCT)

### Visualise Data

#### Line Graphs: Wavelength Tracking

In [ ]:
# Legend used for graph - super manual
legend_labels = ['HCl', 'TEAC', 'TFSA', 'TFA'] if not which_data else \
                ['Eu(EH-Mebip)$_{2}$', 'Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{3}$', 'Cu(EH-Mebip)', 'Cu(EH-Mebip)$_{2}$']

# Designate grouper (to group titrations according to host or guest)
grouper = 'host_name' if not which_data else 'guest_name'

In [ ]:
# Plot data of interest
to_plot = df[['g_h', 'host_name', 'guest_name', 'name', 'value', 'rank', 'wl_track']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
to_plot.sort_values(by='name', key=lambda column: column.map(lambda e: exp_names.index(e)), inplace=True) # Sort dataframe according to index provided

In [ ]:
# Plot data - both complex and decomplex. Legends have to be adjusted manually
for (exp, group) in to_plot.groupby([grouper], as_index=False, sort=False):
    
    fig, ax = plt.subplots()
    
    x_label = '[Guest]:[Zn(EH-Mebip)$_{2}$]' if not which_data else '[HCl]:[M(EH-Mebip)$_{x}$]'
    ax.set_xlabel(x_label)
    ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontsize=20)
    ax.spines[['right', 'top']].set_visible(False)

    # Adjust accordingly
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', hue_order=exp_names, ax=ax, palette='viridis', legend=True)

    # Legend - manual if legend required :( - You will also have to adjust accordingly
    ax.legend(edgecolor='w', framealpha=0)
    for text, label in zip(lnplt.legend_.texts, legend_labels):
        text.set_text(label)

    fig.tight_layout()

#### Spectral Plots: Each Titration (Takes long to plot)

In [ ]:
# Plot data of interest
to_plot = df[['g_h', 'host_name', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]

In [ ]:
# Plot each titration with all the spectra (repeats are included on same graph)
for (exp, group) in to_plot.groupby(['name', grouper], as_index=False, sort=False):
    
    # Cut data - only interested between 250 nm and 600 nm
    group = group.loc[group['Wavelength nm.'].between(250, 600), :]  
    
    fig, ax = plt.subplots(1)

    ax.spines[['right', 'top']].set_visible(False)
    ax.set_xlabel('Wavelength (nm)')
    ax.set_ylabel('Absorbance (a.u.)')
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.2))
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    sns.lineplot(data=group, x="Wavelength nm.", y="Abs.", hue='g_h', palette='flare', ax=ax, legend=True)

    fig.tight_layout()